In [75]:
#Alternative : 
#import files
#from local_navigation import prox
#import class
from thymioV2 import thymio_robot
#import librairies
import time
import numpy as np
import math 

In [ ]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node() #_ = protected #__ = private = shouldn't access node outside of the class
await node.lock()

NodeLockError: Node lock error (current status: busy)

Exception in thread Exception in thread Thread-8:
Traceback (most recent call last):
  File "d:\Programmes\Anaconda\envs\thymio_env\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-6:
Traceback (most recent call last):
  File "d:\Programmes\Anaconda\envs\thymio_env\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-5:
Traceback (most recent call last):
  File "d:\Programmes\Anaconda\envs\thymio_env\lib\threading.py", line 932, in _bootstrap_inner
Thread-7:
Traceback (most recent call last):
  File "d:\Programmes\Anaconda\envs\thymio_env\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "d:\Programmes\Anaconda\envs\thymio_env\lib\site-packages\tdmclient\tcp.py", line 73, in run
    self.run()
  File "d:\Programmes\Anaconda\envs\thymio_env\lib\site-packages\tdmclient\tcp.py", line 73, in run
    self.run()
  File "d:\Programmes\Anaconda\envs\thymio_env\lib\site-packages\tdmclient\tcp.py", line 73, in run
    self.ru

In [ ]:
thymia=thymio_robot()

In [ ]:
thymia.rotate(node,3.1,100)

In [74]:
await node.unlock()

In [71]:
#Thymio control
def motors(l_speed=500, r_speed=500):
    return {
        "motor.left.target": [l_speed],
        "motor.right.target": [r_speed],
    }

async def forward(motor_speed):
    await node.set_variables(motors(motor_speed,motor_speed))

async def rotate(theta, motor_speed): #theta is in radians
    direction_rot=(theta>=0)-(theta<0)
    await node.set_variables(motors(motor_speed*direction_rot, -motor_speed*direction_rot))
    # wait time to get theta 1.44 is the factor to correct
    time=(theta)*100/motor_speed*1.44
    await(client.sleep(time))
    # stop the robot
    await node.set_variables(motors(0, 0))

async def stop_motor():
    await node.set_variables(motors(0,0))

async def get_proximity_values():
    # Wait for the Thymio node
    node = await client.wait_for_node()
    # Wait for the proximity sensor variables
    await node.wait_for_variables({"prox.horizontal"})
    # Get the proximity values : v: Stands for "variables" and is used to access the cached variable values.
    proximity_values = node.v.prox.horizontal
    # Return the value of the front proximity sensor (index 2)
    return proximity_values[0:5]

# Run the asynchronous function and print the result
proximity_values = await get_proximity_values()
print(f"Front Proximity Sensor Value: {proximity_values}")
#await rotate(np.pi,100)


Front Proximity Sensor Value: [3502, 4942, 4905, 4667, 3433]


In [73]:
await stop_motor()

In [72]:
#MAIN CODE : 

#parameters
path_step=0 #This is the index of where we are in the path.
path_coord = np.array([[1,2], [2,3], [3,4]])
b = np.array([3.2, 4.5])
epsilon = 0.5
local_obstacle=False


#to implement in a different way thanks to filtering : 
robot_position=1
end_position=2

async def main():

    global local_obstacle
    while(robot_position!=end_position):
        sens = await get_proximity_values()
        if (sum(sens[i] > threshold_obst for i in range(0, 5)) > 0):
            local_obstacle = True
            start_time = time.time()

        if(local_obstacle):
            sens = await get_proximity_values()
            #sens=list(node["prox.horizontal"])
            await local_navigation()
        
        #if(find_index_path_if_close(path_coord,b,epsilon) and time.time-start_time>2): 
            #path_step=find_index_path_if_close(path_coord, position_robot, epsilon)
            #local_navigation=False

        if not local_obstacle: 
            #Function input : absolute velocity wanted ; output wheel velocity
            #u=speed along x axis
            #v=speed along y axis
            #apply compute wheel velocity to robot in the while loop
            pass

await main()

Turn right
[0, 4003, 4491, 0, 0]
Turn right
[1774, 2861, 1760, 0, 0]
Turn left


CancelledError: 

In [53]:
## Parameters for local navigation
threshold_obst = 3500 
threshold_loc = 2500
local_motor_speed = 100
threshold_obst_list = [3200, 3600, 3600, 3600, 3200]

async def local_navigation():
    threshold_obst = 1000
    threshold_loc = 800
    local_motor_speed = 100
    threshold_obst_list = [3200, 3600, 3600, 3600, 3200]
    sens = await get_proximity_values()

    # Follow the obstacle by the left
    if (sens[0] + sens[1]) > (sens[4] + sens[3]):
        await bypass('right', sens, threshold_loc, local_motor_speed)

    # Follow the obstacle by the right    
    else:
        await bypass('left', sens, threshold_loc, local_motor_speed)

async def bypass(leftright, sens, threshold_loc, local_motor_speed):
    if leftright == "right":
        while sum(sens[i] > threshold_obst for i in range(0, 5)) > 0:
            print("Turn right")
            await rotate(np.pi/6/2, local_motor_speed)
            #await asyncio.sleep(0.2)
            sens = await get_proximity_values()
            print(sens)

        while sens[0] > threshold_loc:
            await forward(local_motor_speed)
            #await asyncio.sleep(0.2)
            #time.sleep(0.2)
            sens = await get_proximity_values()
        
        

    elif leftright == "left":
        while sum(sens[i] > threshold_loc for i in range(0, 5)) > 0:
            print("Turn left")
            await rotate(-np.pi/6, local_motor_speed)
            #await asyncio.sleep(0.2)
            sens = await get_proximity_values()

        while sens[4] > threshold_loc:
            await forward(local_motor_speed)
            #await asyncio.sleep(0.2)
            #time.sleep(0.2)
            sens = await get_proximity_values()

    if(leftright=="right" and sens[0] > threshold_loc):
        await rotate(np.pi/2, 100)
    

    await forward(local_motor_speed)
    #time.sleep(2)
    await stop_motor()

# Run the local_navigation function
#await local_navigation()
#test local navigation





Turn right
Turn left


CancelledError: 

In [ ]:
#find_index if robot abs coord is close enough to a position of the A*paths
def find_index_path_if_close(path_coord, position_robot, epsilon):
    # Displacement vector
    vect_disp = path_coord - position_robot

    # Calculate the distance between the robot's position and the path
    dist = np.sqrt(vect_disp[:, 0]**2 + vect_disp[:, 1]**2)
    
    # Find indices where the distance is less than epsilon
    indices = np.where(dist < epsilon)[0]
    
    if indices.size == 0:
        # No indices found
        #print("No indices found where the distance is less than epsilon.")
        return None
    else:
        # Return the first index
        print(indices)
        return indices[0]

#test
path_coord = np.array([[1,2], [2,3], [3,4]])
b = np.array([3.2, 4.5])
epsilon = 0.5
print(find_index_path_if_close(path_coord,b,epsilon))
if(find_index_path_if_close(path_coord,b,epsilon)==None):
    print(3)

